In [ ]:
!pip install -Uqq 'fastai' 'ddgs'

In [ ]:
from ddgs import DDGS
from fastcore.all import *
import time, json
from fastdownload import download_url # This is for downloading urls
from fastai.vision.all import * # This is for handling and displaying images

def search_images(keywords, max_images=200):
  return L(DDGS().images(keywords, max_results=max_images)).itemgot('image')

In [ ]:
# Gather the training data required for training the model
searches = ('bird','plane')
path = Path('bird_or_plane')

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photos'))
    time.sleep(5)

In [ ]:
# Verify and delete malfunctioning image files
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink)
len(failed)

In [ ]:
# Create Data Blocks for training the model
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

In [ ]:
dls.show_batch(max_n=20)

In [ ]:
#Train the Model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
learn.show_results(max_n=100)

In [ ]:
# Plot top losses
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, nrows=3)

In [ ]:
# Clean data with fastai GUI
from fastai.vision.widgets import *

cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
# Apply the changes
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/'images'/cat)